In [1]:
import numpy as np
import pandas as pd

### Imputation

In [ ]:
pred = np.load('./results/20230712_005035_dm64_df32_el2_topk3_nk3_fq_h_Exp/pred.npy') # (D,L,K)

D, L, K = pred.shape
pred = pred.reshape(D * L, K) # (D*L,K)

# Define the start and end times of each range
ranges_5min = [('05:00:00', '17:55:00')]

# Define the start and end dates for the DatetimeIndex
start_date = '2023-07-01'
end_date = '2023-07-07'

# Create an empty list to store the DatetimeIndex values
index_values = []

# Loop through each date
for date in pd.date_range(start=start_date, end=end_date, freq='D'):
    # Loop through each range and generate the DatetimeIndex values within that range for the current date
    for start_time, end_time in ranges_5min:
        index_values += pd.date_range(start=f'{date.date()} {start_time}', end=f'{date.date()} {end_time}', freq='5min', inclusive='both').tolist()

# Create the DatetimeIndex from the list of values
datetime_5min_test_ids_full = pd.DatetimeIndex(index_values)

pred_df = pd.DataFrame(data=pred, index=datetime_5min_test_ids_full)


# pred_time_range
pred_ranges_5min = [('08:00:00', '08:55:00'), ('12:30:00', '13:25:00'), ('17:00:00', '17:55:00')]

# Define the start and end dates for the DatetimeIndex
start_date = '2023-07-01'
end_date = '2023-07-07'

# Create an empty list to store the DatetimeIndex values
index_values = []

# Loop through each date
for date in pd.date_range(start=start_date, end=end_date, freq='D'):
    # Loop through each range and generate the DatetimeIndex values within that range for the current date
    for start_time, end_time in pred_ranges_5min:
        index_values += pd.date_range(start=f'{date.date()} {start_time}', end=f'{date.date()} {end_time}', freq='5min', inclusive='both').tolist()

datetime_5min_test_ids = pd.DatetimeIndex(index_values)

pred_df = pred_df.loc[datetime_5min_test_ids]

template_df = pd.read_csv('./submit_template.csv',index_col=['Time', 'Week'])

template_df.iloc[:,:] = pred_df.values

template_df.iloc[:, :10] = template_df.iloc[:, :10].astype(int)
template_df.iloc[:, 10:] = template_df.iloc[:, 10:].astype(float).applymap("{:.2f}".format)

template_df.to_csv('./results/submit.csv', index=True)

### Prediction

In [2]:
flow_pred = np.load('./results/20230713_033341_long_term_forecast_flow_dm256_df256_el2_topk5_nk6_fq_h_Exp/pred.npy')
speed_pred = np.load('./results/20230713_033319_long_term_forecast_speed_dm256_df256_el2_topk5_nk6_fq_h_Exp/pred.npy')

In [3]:
D, L, K = flow_pred.shape
flow_pred = flow_pred.reshape(D * L, K) # (D*L,K)
speed_pred = speed_pred.reshape(D * L, K) # (D*L,K)

In [4]:
# flow_pred_df = pd.DataFrame(data=flow_pred, index=datetime_5min_test_ids_full)
# speed_pred_df = pd.DataFrame(data=speed_pred, index=datetime_5min_test_ids_full)

# pred_time_range
pred_ranges_5min = [('08:00:00', '08:55:00'), ('12:30:00', '13:25:00'), ('17:00:00', '17:55:00')]

# Define the start and end dates for the DatetimeIndex
start_date = '2023-07-01'
end_date = '2023-07-07'

# Create an empty list to store the DatetimeIndex values
index_values = []

# Loop through each date
for date in pd.date_range(start=start_date, end=end_date, freq='D'):
    # Loop through each range and generate the DatetimeIndex values within that range for the current date
    for start_time, end_time in pred_ranges_5min:
        index_values += pd.date_range(start=f'{date.date()} {start_time}', end=f'{date.date()} {end_time}', freq='5min', inclusive='both').tolist()

datetime_5min_test_ids = pd.DatetimeIndex(index_values)

flow_pred_df = pd.DataFrame(data=flow_pred, index=datetime_5min_test_ids)
speed_pred_df = pd.DataFrame(data=speed_pred, index=datetime_5min_test_ids)

In [5]:
result_flow_pred = np.sum(flow_pred_df.values.reshape((252,10,4)), axis=2)
result_speed_pred = np.round(np.sum((flow_pred_df.values * speed_pred_df.values).reshape((252,10,4)), axis=2) / result_flow_pred, decimals=2)


template_df = pd.read_csv('./submit_template.csv',index_col=['Time', 'Week'])

num_columns = result_flow_pred.shape[1]


result_df = pd.DataFrame(columns=range(num_columns * 2), index=template_df.index)


for i in range(num_columns):
    result_df.iloc[:, 2 * i] = result_flow_pred[:, i]
    result_df.iloc[:, 2 * i + 1] = result_speed_pred[:, i]

result_df.columns = template_df.columns

num_columns = result_df.shape[1]

for i in range(num_columns):
    if i % 2 == 0:
        result_df.iloc[:, i] = result_df.iloc[:, i].round(0).astype(int)

C:\Users\great\AppData\Local\Temp\ipykernel_15880\580094491.py:14: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  result_df.iloc[:, 2 * i] = result_flow_pred[:, i]
C:\Users\great\AppData\Local\Temp\ipykernel_15880\580094491.py:15: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  result_df.iloc[:, 2 * i + 1] = result_speed_pred[:, i]
C:\Users\great\AppData\Local\Temp\ipykernel_15880\580094491.py:23: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To 

In [6]:
result_df.head(10)

,,Loop1 Flow (Veh/h),Loop1 Speed (km/h),Loop2 Flow (Veh/h),Loop2 Speed (km/h),Loop3 Flow (Veh/h),Loop3 Speed (km/h),Loop4 Flow (Veh/h),Loop4 Speed (km/h),Loop5 Flow (Veh/h),Loop5 Speed (km/h),Loop6 Flow (Veh/h),Loop6 Speed (km/h),Loop7 Flow (Veh/h),Loop7 Speed (km/h),Loop8 Flow (Veh/h),Loop8 Speed (km/h),Loop9 Flow (Veh/h),Loop9 Speed (km/h),Loop10 Flow (Veh/h),Loop10 Speed (km/h)
Time,Week,,,,,,,,,,,,,,,,,,,,
8:00,6,2384,110.790001,2659,109.730003,2591,109.709999,3659,108.459999,2731,109.290001,2804,107.360001,3384,111.379997,3147,105.419998,2914,108.449997,2990,107.199997
8:05,6,2462,110.379997,2511,108.970001,2366,107.320000,3766,108.309998,2497,108.279999,2519,108.400002,3086,112.129997,2806,106.870003,2608,108.900002,2968,104.769997
8:10,6,2721,109.550003,2848,109.519997,2634,108.639999,3844,110.739998,2937,108.309998,2968,107.599998,3533,108.809998,3158,104.690002,3038,106.430000,3217,102.330002
8:15,6,2985,108.860001,3283,109.290001,3186,108.889999,4649,106.220001,3396,110.669998,3353,110.949997,3801,108.440002,3287,104.269997,3136,106.980003,3314,100.910004
8:20,6,2941,110.209999,3213,110.190002,2890,109.650002,4368,108.790001,3142,109.779999,3245,108.010002,3763,108.250000,3524,105.760002,3334,105.050003,3369,103.540001
8:25,6,3087,109.570000,3253,108.519997,3205,107.919998,4438,111.510002,3026,109.129997,3113,108.309998,3494,105.970001,3356,102.629997,3122,108.019997,3389,105.660004
8:30,6,2783,110.250000,3216,109.860001,3004,109.089996,4680,110.349998,2911,108.480003,3220,106.730003,3599,104.860001,3434,104.500000,3223,107.440002,3268,105.209999
8:35,6,3240,110.970001,3426,111.389999,3568,108.940002,4774,107.430000,3328,106.139999,3306,106.139999,3899,108.639999,3686,104.419998,3510,107.940002,3602,105.750000
8:40,6,3547,112.830002,3843,109.889999,3614,108.839996,5151,107.970001,3525,107.550003,3756,106.989998,4342,109.889999,3929,106.760002,3373,107.320000,3608,104.269997


In [7]:
result_df.to_csv('./results/submit.csv', index=True)